# Xây Dựng Với Các Mô Hình Gia Đình Meta

## Giới Thiệu

Bài học này sẽ đề cập đến:

- Khám phá hai mô hình chính của gia đình Meta - Llama 3.1 và Llama 3.2
- Hiểu các trường hợp sử dụng và tình huống phù hợp cho từng mô hình
- Mẫu mã nguồn minh họa các tính năng nổi bật của từng mô hình

## Gia Đình Các Mô Hình Meta

Trong bài học này, chúng ta sẽ tìm hiểu 2 mô hình thuộc gia đình Meta hay còn gọi là "Llama Herd" - Llama 3.1 và Llama 3.2

Những mô hình này có nhiều biến thể khác nhau và có sẵn trên chợ mô hình Github. Dưới đây là thông tin chi tiết hơn về cách sử dụng Github Models để [tạo mẫu với các mô hình AI](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst).

Các biến thể mô hình:
- Llama 3.1 - 70B Instruct
- Llama 3.1 - 405B Instruct
- Llama 3.2 - 11B Vision Instruct
- Llama 3.2 - 90B Vision Instruct

*Lưu ý: Llama 3 cũng có trên Github Models nhưng sẽ không được đề cập trong bài học này*


## Llama 3.1

Với 405 tỷ tham số, Llama 3.1 thuộc nhóm LLM mã nguồn mở.

Phiên bản này là bản nâng cấp so với Llama 3 trước đó với các điểm nổi bật:

- Cửa sổ ngữ cảnh lớn hơn - 128k tokens so với 8k tokens
- Số lượng token đầu ra tối đa lớn hơn - 4096 so với 2048
- Hỗ trợ đa ngôn ngữ tốt hơn - nhờ tăng số lượng token huấn luyện

Những cải tiến này giúp Llama 3.1 xử lý các trường hợp sử dụng phức tạp hơn khi xây dựng ứng dụng GenAI, bao gồm:
- Gọi hàm gốc - khả năng gọi các công cụ và hàm bên ngoài quy trình LLM
- Hiệu suất RAG tốt hơn - nhờ cửa sổ ngữ cảnh lớn hơn
- Tạo dữ liệu tổng hợp - khả năng tạo dữ liệu hiệu quả cho các tác vụ như tinh chỉnh


### Gọi Hàm Gốc

Llama 3.1 đã được tinh chỉnh để hiệu quả hơn trong việc gọi hàm hoặc sử dụng công cụ. Nó cũng có hai công cụ tích hợp sẵn mà mô hình có thể nhận biết khi nào cần sử dụng dựa trên yêu cầu của người dùng. Hai công cụ này là:

- **Brave Search** - Có thể dùng để lấy thông tin mới nhất như thời tiết bằng cách tìm kiếm trên web
- **Wolfram Alpha** - Có thể dùng cho các phép tính toán học phức tạp hơn nên bạn không cần tự viết hàm

Bạn cũng có thể tạo các công cụ tùy chỉnh để LLM có thể gọi.

Trong ví dụ mã dưới đây:

- Chúng ta định nghĩa các công cụ có sẵn (brave_search, wolfram_alpha) trong prompt hệ thống.
- Gửi một prompt từ người dùng hỏi về thời tiết ở một thành phố cụ thể.
- LLM sẽ phản hồi bằng một lệnh gọi công cụ tới Brave Search, ví dụ như sau `<|python_tag|>brave_search.call(query="Stockholm weather")`

*Lưu ý: Ví dụ này chỉ thực hiện lệnh gọi công cụ, nếu bạn muốn lấy kết quả, bạn cần tạo một tài khoản miễn phí trên trang API của Brave và tự định nghĩa hàm*


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import AssistantMessage, SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "meta-llama-3.1-405b-instruct"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)


tool_prompt=f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 23 July 2024

You are a helpful assistant<|eot_id|>
"""

messages = [
    SystemMessage(content=tool_prompt),
    UserMessage(content="What is the weather in Stockholm?"),

]

response = client.complete(messages=messages, model=model_name)

print(response.choices[0].message.content)





### Llama 3.2

Mặc dù là một mô hình ngôn ngữ lớn, Llama 3.1 vẫn còn hạn chế về khả năng đa phương thức. Tức là, chưa thể sử dụng các loại đầu vào khác nhau như hình ảnh để làm prompt và trả về phản hồi. Đây chính là một trong những điểm nổi bật của Llama 3.2. Những tính năng này còn bao gồm:

- Đa phương thức - có thể xử lý cả prompt văn bản lẫn hình ảnh
- Các phiên bản kích thước nhỏ đến vừa (11B và 90B) - giúp linh hoạt trong việc triển khai,
- Các phiên bản chỉ văn bản (1B và 3B) - cho phép triển khai trên thiết bị biên / di động và mang lại độ trễ thấp

Việc hỗ trợ đa phương thức là một bước tiến lớn trong thế giới các mô hình mã nguồn mở. Ví dụ mã dưới đây nhận cả hình ảnh và prompt văn bản để phân tích hình ảnh bằng Llama 3.2 90B.

### Hỗ trợ đa phương thức với Llama 3.2


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "Llama-3.2-90B-Vision-Instruct"


client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(
            content="You are a helpful assistant that describes images in details."
        ),
        UserMessage(
            content=[
                TextContentItem(text="What's in this image?"),
                ImageContentItem(
                    image_url=ImageUrl.load(
                        image_file="sample.jpg",
                        image_format="jpg",
                        detail=ImageDetailLevel.LOW)
                ),
            ],
        ),
    ],
    model=model_name,
)

print(response.choices[0].message.content)

## Việc học không dừng lại ở đây, hãy tiếp tục hành trình

Sau khi hoàn thành bài học này, hãy khám phá [Bộ sưu tập học về AI sinh sinh](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) của chúng tôi để tiếp tục nâng cao kiến thức về AI sinh sinh!



---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn tham khảo chính thức. Đối với các thông tin quan trọng, nên sử dụng dịch vụ dịch thuật chuyên nghiệp bởi con người. Chúng tôi không chịu trách nhiệm đối với bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
